In [1]:
import torch
import torch.nn as nn
import logging
import time
import torch.nn.functional as F

/root/anaconda3/envs/models/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def train(train_iter,dev_iter,test_iter,args,model):
    start_time = time.time()
    steps=0
    logging.info("Training start time:",start_time)
    # 模型、数据放到gpu上。
    # 方法1）：.cuda()   方法2）：.to(device) device是指定好的
    model.cuda()
    # 指定模型的  方式
    model.train()
    # step 1:定义优化器
    optimizer=torch.optim.Adam(model.parameters(),lr=args.learning_rate)
    # step 2:
    for epoch in range(args.epoches):
        logging.info("\n## The {} Epoch, All {} Epochs ! ##".format(epoch, args.epochs))
        # step 3: 读取batch个数据
        for batch in train_iter:
            # 1) 准备数据
            text,label=batch.text,batch.label
            text,label=text.cuda(),label.cuda()
            # 2) 定义损失函数
            lossf=nn.CrossEntropyLoss()
            # 3）梯度归零
            optimizer.zero_grad()
            # 4) 
            logit=model(text)
            # 5) 计算loss
            loss=lossf(logit,label)
            # 6) 
            loss.backward()
            optimizer.step()
            steps+=1
            if steps % args.log_intervel==0:
                train_size = len(train_iter.dataset)
                corrects = (torch.max(logit, 1)[1].view(label.size()).data == label.data).sum()
                accuracy = float(corrects)/batch.batch_size * 100.0
                #是用loss 还是用loss.item?
                logging.info("\n## train_loss {} , train_acc {} ##".format(loss.item(), accuracy))
            if steps % args.eval_interval==0:
                avg_loss,accuracy=eval(args,model,dev_iter)
                logging.info("\n## dev_loss {} , dev_acc {} ##".format(avg_loss, accuracy))
              

In [ ]:
def eval(args,model,eval_iter):
    model.eval()
    loss_avg,corrects=0,0
    size=len(eval_iter)
    for batch in eval_iter:
        feature, target = batch.text, batch.label
        feature.data.t_(), target.data.sub_(1)  # batch first, index align
        logit=model(feature)

        loss=F.cross_entropy(logit,target)
        loss_avg+=loss
        corrects=(torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    size = len(eval_iter.dataset)
    avg_loss = loss.item()/size
    accuracy = 100.0 * float(corrects)/size
    return avg_loss,accuracy

In [ ]:
def test(args,model,test_iter):
    model.load_state_dict(torch.load(args.save_path))
    model.eval()
    # start_time = time.time()
    test_loss, test_acc= eval(args, model, test_iter)
    return test_loss,test_acc